In [1]:
from osgeo import gdal
import fiona
import pandas as pd
from gdalconst import GA_ReadOnly

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
local_path = ''
data_path = local_path + 'MachineLearning/geodata_processing/'

In [3]:
sectors = pd.read_json(data_path + r'vienna_sectors.json', orient='index')
trees = pd.read_json(data_path + r'vienna_trees.json', orient='index')

In [4]:
sectors

,id,sector,upper_left_x,upper_left_y,lower_right_x,lower_right_y
0,0,12/2,16.163534,48.333177,16.197313,48.310740
1,1,13/1,16.197252,48.333222,16.231016,48.310775
2,2,13/2,16.230970,48.333258,16.264719,48.310801
3,3,14/1,16.264688,48.333284,16.298423,48.310817
4,4,14/2,16.298406,48.333300,16.332126,48.310823
...,...,...,...,...,...,...
145,145,57/4,16.500068,48.130843,16.533567,48.108307
146,146,58/3,16.533653,48.130790,16.567138,48.108244
147,147,58/4,16.567239,48.130727,16.600708,48.108172
148,148,59/3,16.600824,48.130655,16.634279,48.108089


In [5]:
# TODO: loop over all sectors if needed
# choose a sector!
sector = '36/1'
sector_name = sector.replace('/', '_') + '_op_2019'
sector_to_split = sectors[sectors['sector'] == sector]
sector_to_split

,id,sector,upper_left_x,upper_left_y,lower_right_x,lower_right_y
67,67,36/1,16.39945,48.243358,16.433067,48.220851


In [6]:
size_x = 1667
size_y = 1667
#Line 1: A: pixel size in the x-direction in map units/pixel
#Line 2: D: rotation about y-axis
#Line 3: B: rotation about x-axis
#Line 4: E: pixel size in the y-direction in map units, almost always negative
#Line 5: C: x-coordinate of the center of the upper left pixel
#Line 6: F: y-coordinate of the center of the upper left pixel
def create_jgw(left_x, upper_y, right_x, lower_y, size_x, size_y):
    
    path = data_path + 'ORTHO/new_jgws/' + sector_name + '_' + str(x_loop) + '_' + str(y_loop) + '.jgw'
    
    newPixelSizeX = abs(right_x - left_x) / size_x
    newPixelSizeY = abs(upper_y - lower_y) / size_y
    #print('Pixel size: {0:0.15f}, -{0:0.14f}'.format(newPixelSizeX, newPixelSizeY))
    
    s = '{0:0.15f}\n'.format(newPixelSizeX)
    s += '{0:0.15f}\n'.format(0)
    s += '{0:0.15f}\n'.format(0)
    s += '-{0:0.14f}\n'.format(newPixelSizeY)
    s += '{0:0.5f}\n'.format(left_x)
    s += '{0:0.5f}'.format(upper_y)
 
    print('\ncreate new jgw file {}:\n{}\n'.format(path, s))
    
    text_file = open(path, 'w')
    text_file.write(s)
    text_file.close()   

In [7]:
# TODO: recalculate size, also using with metadeta for each tree
size = float(75/1667)

def create_txt(output_trees, output_coor_min_x, output_coor_max_y, output_width, output_height):
     
    path = data_path + 'ORTHO/clipped/' + sector_name + '_' + str(x_loop) + '_' + str(y_loop) + '.txt'   
    
    string = ''
    for tree in range(len(output_trees)):
        x = (output_trees.iloc[tree, 4] - output_coor_min_x) / output_width
        y = (output_coor_max_y - output_trees.iloc[tree, 5]) / output_height
        string += '{} {} {} {} {}\r'.format(0, x, y, size, size)
    
    print('create txt file for darknet training {}'.format(path))

    text_file = open(path, 'w')
    text_file.write(string[:-1])
    text_file.close() 

In [8]:
clip_width = 250.0
clip_height = 250.0
def save_clip():
        
    path = data_path + 'ORTHO/' + sector_name + '.jpg'

    data = gdal.Open(path, GA_ReadOnly)
    geoTransform = data.GetGeoTransform()

    # “north up” image without any rotation or shearing
    #adfGeoTransform[0] /* top left x */
    #adfGeoTransform[1] /* w-e pixel resolution */
    #adfGeoTransform[2] /* 0 */
    #adfGeoTransform[3] /* top left y */
    #adfGeoTransform[4] /* 0 */
    #adfGeoTransform[5] /* n-s pixel resolution (negative value) */

    left_x = geoTransform[0]
    upper_y = geoTransform[3]
    right_x = left_x + geoTransform[1] * data.RasterXSize
    lower_y = upper_y + geoTransform[5] * data.RasterYSize
    
    clip_width = abs(right_x - left_x)/loops
    clip_height = abs(upper_y - lower_y)/loops

    clip_left_x = left_x + x_loop * clip_width
    clip_upper_y = upper_y - y_loop * clip_height
    clip_right_x = left_x + (x_loop + 1) * clip_width
    clip_lower_y = upper_y - (y_loop + 1) * clip_height

    print('previous image coordinates {} {} {} {}'.format(clip_left_x, clip_upper_y, clip_right_x, clip_lower_y))

    new_path = data_path + '/ORTHO/clipped/' + sector_name + '_' + str(x_loop) + '_' + str(y_loop) + '.png'
    ds = gdal.Translate(new_path, data, projWin = [clip_left_x, clip_upper_y, clip_right_x, clip_lower_y])
    ds = None
    
    return new_path

## Split the chosen raster

In [9]:
height = abs(float(sector_to_split['upper_left_y']) - float(sector_to_split['lower_right_y']))
width = abs(float(sector_to_split['lower_right_x']) - float(sector_to_split['upper_left_x']))
print('width: {}, height: {}'.format(width, height))

width: 0.033616616599999816, height: 0.022506384799996226


In [11]:
# TODO: loop over all sector splits if needed
# With the loop=10 the image will be clipped into 100 splits, each 0.1 width x 0.1 height of original dimensions

loops = 10
for x_loop in range(loops):
    output_coor_min_x = float(sector_to_split['upper_left_x']) + x_loop * (width/loops)
    output_coor_max_x = float(sector_to_split['upper_left_x']) + (x_loop + 1) * (width/loops)
    
    for y_loop in range(loops):
        print('____________')
        print('x:{}, y:{}, sector:{}'.format(x_loop, y_loop, sector))
        output_coor_max_y = float(sector_to_split['upper_left_y']) - y_loop * (height/loops)
        output_coor_min_y = float(sector_to_split['upper_left_y']) - (y_loop + 1) * (height/loops)

        print('coordinates of the split: {} {} {} {}'.format(output_coor_min_x, output_coor_max_y, output_coor_max_x, output_coor_min_y))
        
        # save a clipped image
        clip_path = save_clip()
        
        # read image size
        data = gdal.Open(clip_path, GA_ReadOnly)
        print(data.RasterXSize, data.RasterYSize)
        
        # create jwg with WGS84 coordinates
        create_jgw(output_coor_min_x, output_coor_max_y, output_coor_max_x, output_coor_min_y, data.RasterXSize, data.RasterYSize)      
        
        tree_indizes = []
        output_trees = trees[(trees['coor_x'] > output_coor_min_x) & (trees['coor_x'] < output_coor_max_x) 
              & (trees['coor_y'] > output_coor_min_y) & (trees['coor_y'] < output_coor_max_y)]
        print('{} trees from {} in the image'.format(len(output_trees), len(trees)))
        
        # create txt for darknet training
        output_width = abs(output_coor_max_x - output_coor_min_x)
        output_height = abs(output_coor_max_y - output_coor_min_y)
        create_txt(output_trees, output_coor_min_x, output_coor_max_y, output_width, output_height)
